In [1]:
import json
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

C:\Users\batt\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
with open ("data.json") as file:
    data=json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Aly ELdeen, your bot assistant',
    "I'm Aly Eldeen, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Aly.', "I'm Aly!", 'Just call me as Aly']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help?',


In [4]:
training_sentences=[]
training_labels=[]
labels=[]
responses=[]
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [5]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service',
 'what is your education ?',
 'what do you do']

In [6]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'complaint',
 'complaint',
 'complaint',
 'education',
 'education']

In [7]:
labels

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint',
 'education']

In [8]:
responses

[['Hello', 'Hi', 'Hi there'],
 ['See you later', 'Have a nice day', 'Bye! Come back again'],
 ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"],
 ['I.m Aly ELdeen, your bot assistant',
  "I'm Aly Eldeen, an Artificial Intelligent bot"],
 ['You can call me Aly.', "I'm Aly!", 'Just call me as Aly'],
 ['Tell me how can assist you',
  'Tell me your problem to assist you',
  'Yes Sure, How can I support you'],
 ['You can just easily create a new account from our web site',
  'Just go to our web site and follow the guidelines to create a new account'],
 ['Please provide us your complaint in order to assist you',
  'Please mention your complaint, we will reach you and sorry for any inconvenience caused'],
 ["I'm a machine learning engineer graduated from ain shams university",
  "I'm a mechatronics engineer works as a machine learning engineer who is graduated from ain shams university at 2022"]]

In [9]:
label_encoder=LabelEncoder()
label_encoder.fit(training_labels)
training_labels_encoded=label_encoder.transform(training_labels)
training_labels_encoded

array([5, 5, 5, 5, 5, 4, 4, 4, 8, 8, 8, 8, 0, 0, 0, 7, 7, 7, 6, 6, 6, 6,
       6, 6, 6, 2, 2, 2, 2, 2, 1, 1, 1, 3, 3])

In [10]:
vocab_size=1000
embedding_dim=16
max_len=20
tokenizer=Tokenizer(num_words=vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(training_sentences)
padded_sequences=pad_sequences(sequences,truncating='post',maxlen=max_len)

In [11]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()
epochs=500

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

In [12]:
history=model.fit(padded_sequences,np.array(training_labels_encoded),epochs=epochs)

Epoch 1/500
2/2 [==============================] - 16s 43ms/step - loss: 2.1967 - accuracy: 0.0857
Epoch 2/500
2/2 [==============================] - 0s 47ms/step - loss: 2.1952 - accuracy: 0.0857
Epoch 3/500
2/2 [==============================] - 0s 31ms/step - loss: 2.1948 - accuracy: 0.0857
Epoch 4/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1940 - accuracy: 0.2571
Epoch 5/500
2/2 [==============================] - 0s 31ms/step - loss: 2.1940 - accuracy: 0.1714
Epoch 6/500
2/2 [==============================] - 0s 31ms/step - loss: 2.1939 - accuracy: 0.3429
Epoch 7/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1937 - accuracy: 0.2857
Epoch 8/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1933 - accuracy: 0.3143
Epoch 9/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1929 - accuracy: 0.3429
Epoch 10/500
2/2 [==============================] - 0s 32ms/step - loss: 2.1924 - accuracy: 0.4286
Epoch 11/500
2/2 [

2/2 [==============================] - 0s 31ms/step - loss: 2.0975 - accuracy: 0.3143
Epoch 84/500
2/2 [==============================] - 0s 36ms/step - loss: 2.0948 - accuracy: 0.3143
Epoch 85/500
2/2 [==============================] - 0s 33ms/step - loss: 2.0921 - accuracy: 0.3429
Epoch 86/500
2/2 [==============================] - 0s 19ms/step - loss: 2.0890 - accuracy: 0.3429
Epoch 87/500
2/2 [==============================] - 0s 41ms/step - loss: 2.0859 - accuracy: 0.3429
Epoch 88/500
2/2 [==============================] - 0s 30ms/step - loss: 2.0829 - accuracy: 0.3429
Epoch 89/500
2/2 [==============================] - 0s 26ms/step - loss: 2.0795 - accuracy: 0.3429
Epoch 90/500
2/2 [==============================] - 0s 29ms/step - loss: 2.0765 - accuracy: 0.3429
Epoch 91/500
2/2 [==============================] - 0s 28ms/step - loss: 2.0733 - accuracy: 0.3429
Epoch 92/500
2/2 [==============================] - 0s 29ms/step - loss: 2.0704 - accuracy: 0.3429
Epoch 93/500
2/2 [=====

2/2 [==============================] - 0s 49ms/step - loss: 1.6419 - accuracy: 0.3714
Epoch 166/500
2/2 [==============================] - 0s 36ms/step - loss: 1.6353 - accuracy: 0.3714
Epoch 167/500
2/2 [==============================] - 0s 33ms/step - loss: 1.6292 - accuracy: 0.3714
Epoch 168/500
2/2 [==============================] - 0s 21ms/step - loss: 1.6225 - accuracy: 0.3714
Epoch 169/500
2/2 [==============================] - 0s 25ms/step - loss: 1.6159 - accuracy: 0.3714
Epoch 170/500
2/2 [==============================] - 0s 20ms/step - loss: 1.6092 - accuracy: 0.3714
Epoch 171/500
2/2 [==============================] - 0s 21ms/step - loss: 1.6031 - accuracy: 0.3714
Epoch 172/500
2/2 [==============================] - 0s 20ms/step - loss: 1.5959 - accuracy: 0.3714
Epoch 173/500
2/2 [==============================] - 0s 21ms/step - loss: 1.5911 - accuracy: 0.3714
Epoch 174/500
2/2 [==============================] - 0s 19ms/step - loss: 1.5871 - accuracy: 0.4000
Epoch 175/500


2/2 [==============================] - 0s 18ms/step - loss: 1.1853 - accuracy: 0.6000
Epoch 248/500
2/2 [==============================] - 0s 20ms/step - loss: 1.1815 - accuracy: 0.6000
Epoch 249/500
2/2 [==============================] - 0s 20ms/step - loss: 1.1769 - accuracy: 0.6286
Epoch 250/500
2/2 [==============================] - 0s 21ms/step - loss: 1.1715 - accuracy: 0.6286
Epoch 251/500
2/2 [==============================] - 0s 20ms/step - loss: 1.1656 - accuracy: 0.6286
Epoch 252/500
2/2 [==============================] - 0s 19ms/step - loss: 1.1593 - accuracy: 0.6286
Epoch 253/500
2/2 [==============================] - 0s 20ms/step - loss: 1.1528 - accuracy: 0.6286
Epoch 254/500
2/2 [==============================] - 0s 19ms/step - loss: 1.1473 - accuracy: 0.6286
Epoch 255/500
2/2 [==============================] - 0s 20ms/step - loss: 1.1418 - accuracy: 0.6286
Epoch 256/500
2/2 [==============================] - 0s 18ms/step - loss: 1.1367 - accuracy: 0.6571
Epoch 257/500


2/2 [==============================] - 0s 17ms/step - loss: 0.8361 - accuracy: 0.6571
Epoch 330/500
2/2 [==============================] - 0s 20ms/step - loss: 0.8325 - accuracy: 0.6571
Epoch 331/500
2/2 [==============================] - 0s 18ms/step - loss: 0.8298 - accuracy: 0.6571
Epoch 332/500
2/2 [==============================] - 0s 17ms/step - loss: 0.8250 - accuracy: 0.6571
Epoch 333/500
2/2 [==============================] - 0s 21ms/step - loss: 0.8216 - accuracy: 0.6571
Epoch 334/500
2/2 [==============================] - 0s 23ms/step - loss: 0.8198 - accuracy: 0.6571
Epoch 335/500
2/2 [==============================] - 0s 31ms/step - loss: 0.8181 - accuracy: 0.6571
Epoch 336/500
2/2 [==============================] - 0s 23ms/step - loss: 0.8160 - accuracy: 0.6571
Epoch 337/500
2/2 [==============================] - 0s 23ms/step - loss: 0.8140 - accuracy: 0.6571
Epoch 338/500
2/2 [==============================] - 0s 20ms/step - loss: 0.8104 - accuracy: 0.6571
Epoch 339/500


2/2 [==============================] - 0s 16ms/step - loss: 0.5981 - accuracy: 0.7429
Epoch 412/500
2/2 [==============================] - 0s 16ms/step - loss: 0.5968 - accuracy: 0.8000
Epoch 413/500
2/2 [==============================] - 0s 16ms/step - loss: 0.5934 - accuracy: 0.8286
Epoch 414/500
2/2 [==============================] - 0s 31ms/step - loss: 0.5913 - accuracy: 0.8286
Epoch 415/500
2/2 [==============================] - 0s 31ms/step - loss: 0.5881 - accuracy: 0.8286
Epoch 416/500
2/2 [==============================] - 0s 31ms/step - loss: 0.5851 - accuracy: 0.8286
Epoch 417/500
2/2 [==============================] - 0s 16ms/step - loss: 0.5831 - accuracy: 0.8286
Epoch 418/500
2/2 [==============================] - 0s 31ms/step - loss: 0.5801 - accuracy: 0.8286
Epoch 419/500
2/2 [==============================] - 0s 31ms/step - loss: 0.5772 - accuracy: 0.8286
Epoch 420/500
2/2 [==============================] - 0s 16ms/step - loss: 0.5739 - accuracy: 0.8286
Epoch 421/500


2/2 [==============================] - 0s 16ms/step - loss: 0.4155 - accuracy: 0.9143
Epoch 494/500
2/2 [==============================] - 0s 16ms/step - loss: 0.4129 - accuracy: 0.9143
Epoch 495/500
2/2 [==============================] - 0s 31ms/step - loss: 0.4095 - accuracy: 0.9143
Epoch 496/500
2/2 [==============================] - 0s 31ms/step - loss: 0.4065 - accuracy: 0.9143
Epoch 497/500
2/2 [==============================] - 0s 16ms/step - loss: 0.4052 - accuracy: 0.9143
Epoch 498/500
2/2 [==============================] - 0s 19ms/step - loss: 0.4030 - accuracy: 0.9143
Epoch 499/500
2/2 [==============================] - 0s 31ms/step - loss: 0.3999 - accuracy: 0.9143
Epoch 500/500
2/2 [==============================] - 0s 16ms/step - loss: 0.3946 - accuracy: 0.9143


In [13]:
model.save("Model")
with open("tokenizer.pickle","wb") as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)
with open("label_encoder.pickle","wb") as handle:
    pickle.dump(label_encoder,handle,protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: Model\assets


In [14]:
with open ('data.json') as file:
    data=json.load(file)

In [15]:
def chat():
    print("Start messaging with the bot (type quit to stop)!")
    model=tf.keras.models.load_model("Model")
    with open("tokenizer.pickle","rb") as handle:
        tokenizer=pickle.load(handle)
    with open("label_encoder.pickle","rb") as handle:
        label_encoder=pickle.load(handle)
    max_len=20
    while(True):
        print("User: ")
        inp = input()
        if inp.lower() == "quit":
            break
        result=model.predict(tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post',maxlen=max_len))
        tag=label_encoder.inverse_transform([np.argmax(result)])
        for i in data['intents']:
            if i['tag']==tag:
                print("ChatBot:",np.random.choice(i['responses']))

In [ ]:
chat